In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [5]:
def f(series):
    return series.shift(1).cumsum().fillna(0) / np.arange(len(series))


# prev_content_type_id

In [6]:
df["target_enc_content_id"] = df.groupby("content_id")["answered_correctly"].transform(f)

In [8]:
df["prev_content_type_id"] = df.groupby("user_id")["content_type_id"].shift(1).fillna(-1)

In [9]:
df.groupby("prev_content_type_id")["answered_correctly"].describe()

,count,mean,std,min,25%,50%,75%,max
prev_content_type_id,,,,,,,,
-1.0,39277.0,0.681951,0.465724,0.0,0.0,1.0,1.0,1.0
0.0,9607898.0,0.653471,0.475864,0.0,0.0,1.0,1.0,1.0
1.0,190367.0,0.727542,0.445225,0.0,0.0,1.0,1.0,1.0


# prev_content_id

In [25]:
df["prev_content_id"] = df.groupby("user_id")["content_id"].shift(1).fillna(-1)

In [31]:
df["content_id_ary"] = [str(x) if x[2] == 1 else np.nan for x in df[["content_id", "prev_content_id", "prev_content_type_id"]].values.tolist()]

In [32]:
ary_map = df.groupby("content_id_ary").size()

In [33]:
df["content_id_ary_countenc"] = df["content_id_ary"].map(ary_map)

In [36]:
df.groupby("content_id_ary_countenc")["answered_correctly"].describe().head(30)

,count,mean,std,min,25%,50%,75%,max
content_id_ary_countenc,,,,,,,,
1.0,41880.0,0.627292,0.483531,0.0,0.00,1.0,1.0,1.0
2.0,9507.0,0.687283,0.463624,0.0,0.00,1.0,1.0,1.0
3.0,5914.0,0.733176,0.442337,0.0,0.00,1.0,1.0,1.0
4.0,4694.0,0.742437,0.437339,0.0,0.00,1.0,1.0,1.0
5.0,4204.0,0.752854,0.431403,0.0,1.00,1.0,1.0,1.0
6.0,3864.0,0.747412,0.434553,0.0,0.00,1.0,1.0,1.0
7.0,3374.0,0.772970,0.418974,0.0,1.00,1.0,1.0,1.0
8.0,3312.0,0.754529,0.430431,0.0,1.00,1.0,1.0,1.0
9.0,2826.0,0.768577,0.421816,0.0,1.00,1.0,1.0,1.0


In [ ]:
df.groupby("content_id_ary_countenc")["answered_correctly"].describe().head(30)